# Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter
from better_profanity import profanity
from textblob import TextBlob 
import nltk
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
filter_name = 'weather since:2021-02-24 until:2023-02-21 exclude:retweets exclude:replies lang:en'

scraper = sntwitter.TwitterSearchScraper(filter_name)
scraper
tweets = []
n_tweets = 100
for i, tweet in tqdm(enumerate(scraper.get_items()), total = n_tweets):
    data= [
        tweet.id,
        tweet.date,
        tweet.rawContent,
        tweet.user.username,
        tweet.likeCount,
        tweet.retweetCount,
        tweet.replyCount,
        tweet.lang,
        tweet.user.location
    ]
#     if tweet.lang == 'en':
    tweets.append(data)
    if i > n_tweets:
        break
tweet_df2 = pd.DataFrame(tweets, columns = ['id','datetime','rawcontent','username','likeCount','retweetCount','replyCount','language','location'])

  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
(tweet_df2['language'] == 'en').count()

102

In [7]:
tweet_df2.sample(5)

,id,datetime,rawcontent,username,likeCount,retweetCount,replyCount,language,location
71,1627819384696672256,2023-02-20 23:55:59+00:00,"FINAL Prediction for Tuesday, February 21, 202...",VWeatherWatcher,14,1,2,en,Northeastern Pennsylvania
91,1627819164445310977,2023-02-20 23:55:06+00:00,"Hey Europe famjam, anybody wanna lemme know wt...",DinosaurChurch,2,1,5,en,Canada
23,1627820151088332801,2023-02-20 23:59:02+00:00,20/02/23 23:59\nCurrent conditions: Temp: 8.0C...,CM3Weather,0,0,0,en,"Little Waltham, CM3, England"
58,1627819620987011075,2023-02-20 23:56:55+00:00,Need this one tonight!,GaudetWeather,2,0,0,en,"Lexington, Kentucky"
65,1627819534752047105,2023-02-20 23:56:35+00:00,It was like this in my local @morrisons today....,SooozeKerr,0,2,0,en,"Suffolk, UK."


# Data Preprocessing

In [8]:
tweet_df = tweet_df2.copy()
tweet_df

,id,datetime,rawcontent,username,likeCount,retweetCount,replyCount,language,location
0,1627820372698562570,2023-02-20 23:59:54+00:00,"Sunset for February 20, 2023 at 05:31PM! Have ...",GallatinWeather,0,0,0,en,
1,1627820366725894144,2023-02-20 23:59:53+00:00,"Report at 00:00 \rWind:SW at av. 0mph, max gus...",HBayWeather,0,0,0,en,Sunny Herne Bay (the UK one)
2,1627820355183075328,2023-02-20 23:59:50+00:00,I think I froze half my piercings off in this ...,gymlifeanimal,200,3,37,en,On a beach somewhere
3,1627820344923783171,2023-02-20 23:59:48+00:00,Your weather for tomorrow @denverchannel https...,MikeNelson247,17,1,1,en,"Denver, CO"
4,1627820342084268034,2023-02-20 23:59:47+00:00,"9 years ago today in Breckinridge, Colorado! I...",GamecockSmoke,1,0,0,en,
...,...,...,...,...,...,...,...,...,...
97,1627819146028146693,2023-02-20 23:55:02+00:00,"Wichita Weather 61F at 5:55 PM, Wind WNW 1 Gus...",SouthWindWX,0,0,0,en,
98,1627819141544419331,2023-02-20 23:55:01+00:00,"Dixon, MO At: 5:55 PM Temp: 57.4°F Visibility:...",dixonweather,0,0,0,en,"Dixon, Missouri USA"
99,1627819138252001280,2023-02-20 23:55:00+00:00,"Wind 5 mph ENE. Barometer 1017.71 hPa, Falling...",quarmbyweather,0,0,0,en,Huddersfield
100,1627819128370110465,2023-02-20 23:54:58+00:00,There is a Fire Weather Watch on Tuesday and a...,KCBD11,0,0,0,en,"Lubbock, TX"


## Convert datetime to date

In [9]:
tweet_df.insert(2, 'date', pd.to_datetime(tweet_df['datetime']).dt.strftime('%Y-%m-%d'))
tweet_df.drop('datetime',axis=1,inplace=True)
tweet_df.sample(5)

,id,date,rawcontent,username,likeCount,retweetCount,replyCount,language,location
70,1627819419589087232,2023-02-20,Tracking Temps in #ATX. Here's how today compa...,ChikageWeather,2,0,0,en,"Austin, TX"
38,1627819935450927104,2023-02-20,A series of passing disturbances will keep uns...,nwsjacksonky,2,1,0,en,"Jackson, KY"
46,1627819812981207040,2023-02-20,"Right now: Smoke, Temperature: 21.02C, Humidit...",WeatherAmdavad,0,0,0,en,Ahmedabad
89,1627819189313347585,2023-02-20,Delhi Weather: Delhi’s heat broke a 17-year re...,DEENEWS_IN,0,0,0,en,
41,1627819916257640448,2023-02-20,I can’t comment on whether it’s the weather or...,kentbuse,0,0,1,en,"Cambridge, England"


## save as csv

In [10]:
tweet_df.to_csv(f"dataset/{'weather'}.csv", index = False)

## import scraped csv

In [12]:
w_df = pd.read_csv(f"dataset/{'weather'}.csv")
w_df

,id,date,rawcontent,username,likeCount,retweetCount,replyCount,language,location
0,1627820372698562570,2023-02-20,"Sunset for February 20, 2023 at 05:31PM! Have ...",GallatinWeather,0,0,0,en,NaN
1,1627820371607945217,2023-02-20,I guess our govt decided to stop playing whack...,fusion_allusion,0,0,0,en,NaN
2,1627820366725894144,2023-02-20,"Report at 00:00 \rWind:SW at av. 0mph, max gus...",HBayWeather,0,0,0,en,Sunny Herne Bay (the UK one)
3,1627820355183075328,2023-02-20,I think I froze half my piercings off in this ...,gymlifeanimal,197,2,37,en,On a beach somewhere
4,1627820344923783171,2023-02-20,Your weather for tomorrow @denverchannel https...,MikeNelson247,17,1,1,en,"Denver, CO"
...,...,...,...,...,...,...,...,...,...
97,1627819151799767040,2023-02-20,"Weather-Frankston, Australia @ 10:55 Temp-19.1...",frankstonwethr,0,0,0,en,NaN
98,1627819146028146693,2023-02-20,"Wichita Weather 61F at 5:55 PM, Wind WNW 1 Gus...",SouthWindWX,0,0,0,en,NaN
99,1627819141544419331,2023-02-20,"Dixon, MO At: 5:55 PM Temp: 57.4°F Visibility:...",dixonweather,0,0,0,en,"Dixon, Missouri USA"
100,1627819138252001280,2023-02-20,"Wind 5 mph ENE. Barometer 1017.71 hPa, Falling...",quarmbyweather,0,0,0,en,Huddersfield


## Check for null values

In [13]:
w_df.isnull().sum()

id               0
date             0
rawcontent       0
username         0
likeCount        0
retweetCount     0
replyCount       0
language         0
location        22
dtype: int64

## Performing EDA

In [14]:
w_df.loc[w_df['likeCount']>=10]

,id,date,rawcontent,username,likeCount,retweetCount,replyCount,language,location
3,1627820355183075328,2023-02-20,I think I froze half my piercings off in this ...,gymlifeanimal,197,2,37,en,On a beach somewhere
4,1627820344923783171,2023-02-20,Your weather for tomorrow @denverchannel https...,MikeNelson247,17,1,1,en,"Denver, CO"
10,1627820303844868097,2023-02-20,Peaceful sunset over Morse after a wonderful M...,MorseLakeWeathe,23,0,2,en,Hamilton County Indiana
19,1627820184324001792,2023-02-20,Winds increase late Tuesday with gusts to 40 p...,MandyBaileyWX,10,3,0,en,Philippians 3:7-8
38,1627819938789441545,2023-02-20,Significant coast-to-coast storm will sweep ac...,Beginning22End,13,2,5,en,Rumble: pray4america
40,1627819933492219904,2023-02-20,A very strong weather system will produce pote...,JoeKOAT,12,3,0,en,"Albuquerque, NM"
43,1627819909303377920,2023-02-20,Massive #winterstorm. It covers basically the ...,tymetwx,10,2,0,en,NaN
65,1627819550027939840,2023-02-20,#goodmorning #Beijing \nwhat a beautiful weath...,ChanJoe18,14,2,1,en,NaN
69,1627819464594006021,2023-02-20,I need the weather to pick a side because this...,YuneKokoro,32,0,3,en,The Dark Alleyway
72,1627819384696672256,2023-02-20,"FINAL Prediction for Tuesday, February 21, 202...",VWeatherWatcher,14,1,2,en,Northeastern Pennsylvania


In [15]:
w_df.loc[(w_df['likeCount']>4)&(w_df['retweetCount']>2)]

,id,date,rawcontent,username,likeCount,retweetCount,replyCount,language,location
19,1627820184324001792,2023-02-20,Winds increase late Tuesday with gusts to 40 p...,MandyBaileyWX,10,3,0,en,Philippians 3:7-8
40,1627819933492219904,2023-02-20,A very strong weather system will produce pote...,JoeKOAT,12,3,0,en,"Albuquerque, NM"
79,1627819294250905603,2023-02-20,A late February weather system blowing into We...,DougPigsleyKOMO,20,4,0,en,NaN
89,1627819200835092481,2023-02-20,#ONStorm #ONwx Good evening! We've got a quick...,IWeatherON,270,67,18,en,"Ontario, Canada"


# Cleaning the tweets

In [16]:
tweet_list = w_df.rawcontent.to_list()

In [17]:
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    r = tweet.lower()
    r = profanity.censor(r)
    r = re.sub("'", "", r) # This is to avoid removing contractions in english
    r = re.sub("@[A-Za-z0-9_]+","", r) # remove mentions
    r = re.sub("#[A-Za-z0-9_]+","", r) # remove hashtags
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r